In [ ]:
# Library Imports
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments, set_seed
from datasets import *
import numpy as np
import torch
import ast

torch.cuda.empty_cache()

In [ ]:
# Seeds used for uncertainty estimation
# seeds = [42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52]

# Set the seed
SEED = 46
set_seed(SEED)

# Any combination of these years and dataset types can be used
# year = 2020
# year = 2022
# dataset_type = "full"
# dataset_type = "smaller"

# Only the combination of this year and dataset type can be used
year = 2022
dataset_type = "handwritten"

# Only combinations of this year and dataset types can be used
# year = 2042 # idk some random number for file names
# dataset_type = "full_combined"
# dataset_type = "smaller_combined"

# Standard path
# local_models_path = '../models/T5' 
# Path for SE models
local_models_path = '../models/T5/SE'

# Small model
model_name = "mrm8488/t5-small-finetuned-squadv2" 
# Base model
# model_name = "mrm8488/t5-base-finetuned-squadv2"

Reference notebooks:

https://colab.research.google.com/github/patil-suraj/exploring-T5/blob/master/T5_on_TPU.ipynb#scrollTo=KdmKlMkfcLa0

https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/summarization.ipynb

### Load tokenizer and model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

### Load the dataset

In [ ]:
# Load the dataset from file and split it into train and test datasets
if dataset_type == "full":
    data = load_dataset('csv', data_files=f"../../data/clean/sustainability-report-{year}-squad-format.csv",
                        delimiter=";", split='train').train_test_split(test_size=0.3, shuffle=True, seed=SEED)
elif dataset_type == "smaller":
    data = load_dataset('csv', data_files=f"../../data/clean/sustainability-report-{year}-squad-format.csv",
                        delimiter=";", split='train').train_test_split(test_size=0.3, shuffle=True, seed=SEED)
    data["train"] = data["train"].select(range(len(data["train"]) // 2))
elif dataset_type == "full_combined":
    data = load_dataset('csv', data_files="../../data/clean/sustainability-report-2042-squad-format.csv",
                        delimiter=";", split="train").train_test_split(test_size=0.3, shuffle=True, seed=SEED)
elif dataset_type == "smaller_combined":
    data = load_dataset('csv', data_files="../../data/clean/sustainability-report-2042-squad-format.csv",
                        delimiter=";", split="train").train_test_split(test_size=0.3, shuffle=True, seed=SEED)
    data["train"] = data["train"].select(range(len(data["train"]) // 2))
elif dataset_type == "handwritten":
    data = load_dataset('csv', data_files=f"../../data/clean/QA_SR_2022_Expert-squad-format.csv",
                        delimiter=";", split='train').train_test_split(test_size=0.3, shuffle=True, seed=SEED)
else:
    raise Exception("Invalid dataset type")

In [ ]:
# Reformat the train and test set such as they adhere to the SQuAD format (reading from cvs loads strings not objects as expected)
data["test"] = data["test"].map(
    lambda example: ast.literal_eval(example["answers"]))
data["test"] = data["test"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {
                                "text": example["text"], "answer_start": example["answer_start"]}})
# replace all "\n" with " " in the context, answers and questions
data["test"] = data["test"].map(lambda example: {"question": example["question"].replace("\n", " "), "context": example["context"].replace("\n", " "), "answers": {
                                "text": [example["answers"]["text"][0].replace("\n", " ")], "answer_start": example["answers"]["answer_start"]}})
data["test"] = data["test"].remove_columns(["text", "answer_start"])

data["train"] = data["train"].map(
    lambda example: ast.literal_eval(example["answers"]))
data["train"] = data["train"].map(lambda example: {"question": example["question"], "context": example["context"], "answers": {
                                  "text": example["text"], "answer_start": example["answer_start"]}})
# replace all "\n" with " " in the context, answers and questions
data["train"] = data["train"].map(lambda example: {"question": example["question"].replace("\n", " "), "context": example["context"].replace("\n", " "), "answers": {
                                "text": [example["answers"]["text"][0].replace("\n", " ")], "answer_start": example["answers"]["answer_start"]}})
data["train"] = data["train"].remove_columns(["text", "answer_start"])

### Tokenize the dataset

In [ ]:
# process the examples in input and target text format and the eos token at the end
def add_eos_to_examples(example):
    example['input_text'] = 'question: %s  context: %s </s>' % (
        example['question'], example['context'])
    example['target_text'] = '%s </s>' % example['answers']['text'][0]
    return example

# tokenize the examples
def convert_to_features(examples):
    model_inputs = tokenizer(
        examples['input_text'], pad_to_max_length=True, max_length=512, truncation=True)

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples['target_text'], pad_to_max_length=True, max_length=128, truncation=True)
        temp = np.array(labels["input_ids"])
        temp[temp == tokenizer.pad_token_id] = -100
        labels["input_ids"] = temp.tolist()

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
train_data, test_data = data["train"], data["test"]

train_data = train_data.map(add_eos_to_examples, load_from_cache_file=False)
train_data = train_data.map(
    convert_to_features, batched=True, load_from_cache_file=False)

test_data = test_data.map(add_eos_to_examples, load_from_cache_file=False)
test_data = test_data.map(
    convert_to_features, batched=True, load_from_cache_file=False)

### Fine tunning

In [ ]:
name = model_name.split("/")[-1]
if "SE" in local_models_path:
    output_dir = f"{local_models_path}/{name}-finetuned-NLB-QA-{year}-{dataset_type}-{SEED}"
else:
    output_dir = f"{local_models_path}/{name}-finetuned-NLB-QA-{year}-{dataset_type}"
if "small" in model_name:
    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=25,
        predict_with_generate=True,
        fp16=True,
        push_to_hub=False,
        load_best_model_at_end=True
    )
elif "base" in model_name:
    training_args = Seq2SeqTrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=25,
        predict_with_generate=True,
        fp16=True,
        push_to_hub=False,
        load_best_model_at_end=True
    )
else:
    raise ValueError("Model name not supported")

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()
trainer.save_model(output_dir)